# Compute Kinetic Energy anomaly trends (as computed by Hu et al. 2020)

In [6]:
# Import libraries
import xarray as xr
import cmocean as cm
import cartopy.crs as ccrs

import pylab as plt
import numpy as np
# Inline plotting
%matplotlib inline

In [7]:
from dask.distributed import Client

In [8]:
from utils import area,ccrs_land,add_patches
import datetime as datetime

In [9]:
c = Client()
c

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-20.10/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37889 instead
  warnings.warn(


Client Scheduler: tcp://127.0.0.1:32821 Dashboard: /proxy/37889/status,Cluster Workers: 8 Cores: 48 Memory: 202.48 GB


In [10]:
files='/g/data/ua8/CMEMS_SeaLevel/v2019/*/*.nc'

In [11]:
dataset_AVISO = xr.open_mfdataset(files,parallel=True)

In [12]:
dataset_AVISO = dataset_AVISO.rename({'latitude':'lat','longitude':'lon'})

In [14]:
ugos = dataset_AVISO.ugos.chunk({"lat": 100, "lon": 100,'time':357})
vgos = dataset_AVISO.ugos.chunk({"lat": 100, "lon": 100,'time':357})

In [15]:
KE = 1/2 * (ugos**2 + vgos**2).coarsen({'lat':4,'lon':4}).mean().compute()

In [16]:
# Use same reference mean (1993-2012)
mean_KE = KE.sel(time=slice('1993','2012')).mean()

In [17]:
KE_anomaly = (KE - mean_KE)

In [18]:
KE_rolled = KE_anomaly.rolling(time=365,center=True).mean().compute()

In [19]:
KE_rolled = KE_rolled.to_dataset(name="KE'")

In [20]:
KE_rolled.attrs['title'] = "Kinetic Energy anomaly"
KE_rolled.attrs['Description'] = """Kinetic Energy anomaly computed from AVISO+ altimetry products (Satellite reproduction of Hu's et al. (2020) paper)."""
KE_rolled.attrs['Publication'] = "Dataset created for Martínez-Moreno, J. et. al. 2020: \n 'Global changes in oceanic mesoscale currents over the satellite altimetry record'"
KE_rolled.attrs['Author'] = "Josué Martínez-Moreno"
KE_rolled.attrs['Contact'] = "josue.martinezmoreno@anu.edu.au"

KE_rolled.attrs['Created date'] = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")

counter = 0
units = [r"$m^2s^{-2}$"]
names = ["KE'"]
long_names = ["Kinetic Energy anomaly"]

KE_rolled["KE'"].attrs['units'] = units[counter]
KE_rolled["KE'"].attrs['name'] = names[counter]
KE_rolled["KE'"].attrs['long_name'] = long_names[counter]

KE_rolled["KE'"].attrs['missing_value'] = np.nan
KE_rolled["KE'"].attrs['valid_min'] = np.nanmin(KE_rolled["KE'"])
KE_rolled["KE'"].attrs['valid_max'] = np.nanmax(KE_rolled["KE'"])
KE_rolled["KE'"].attrs['valid_range'] = [np.nanmin(KE_rolled["KE'"]),np.nanmax(KE_rolled["KE'"])]

In [21]:
comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in KE_rolled.data_vars}

KE_rolled.to_netcdf('../datasets/KE_anomaly_timeseries.nc', encoding=encoding)